# 1 Use existing functions

In [21]:
!pip install line_profiler
!pip install cupy-cuda112

In [3]:
%load_ext line_profiler

In [2]:
from cupyx.profiler import benchmark
import numpy as np
from scipy.signal import fftconvolve, convolve
import cupy
from cupyx.scipy import ndimage
import sys
sys.path.insert(1, 'C:/Users/jo77pihe/Documents/MasterThesis_OfSpinesAndDendrites')
import deconv.utils.c_convolve as cc

In [4]:
a128 = np.random.rand(10,128,128).astype(np.float32)
b128 = np.random.rand(10,128,128).astype(np.float32)
a256 = np.random.rand(20,256,256).astype(np.float32)
b256 = np.random.rand(20,256,256).astype(np.float32)
a512_1 = np.random.rand(20,512,512).astype(np.float32)
b512_1 = np.random.rand(20,512,512).astype(np.float32)
a512_2 = np.random.rand(40,512,512).astype(np.float32)
b512_2 = np.random.rand(40,512,512).astype(np.float32)
a512_3 = np.random.rand(80,512,512).astype(np.float32)
b512_3 = np.random.rand(80,512,512).astype(np.float32)

## 1.0 Numpy
* Only supports 1D convolution

## 1.1: Scipy

In [5]:
def sci_conv(a, b, n = 1):
    for _ in range(n):
        a = convolve(a,b, mode='same')
    return a


def sci_fftconv(a, b, n = 1):
    for _ in range(n):
        a = fftconvolve(a,b, mode='same')
    return a

In [9]:
con_res = np.zeros((5,12))
con_std = np.zeros((5,12))
n_rep = 105
n_conv = 14

x = benchmark(sci_conv, (a128,b128,n_conv), n_repeat=n_rep)
con_res[0,0] = np.mean(x.cpu_times)
con_res[0,1] = np.mean(x.gpu_times)
con_std[0,0] = np.std(x.cpu_times)
con_std[0,1] = np.std(x.gpu_times)
x = benchmark(sci_conv, (a256,b256,n_conv), n_repeat=n_rep)
con_res[1,0] = np.mean(x.cpu_times)
con_res[1,1] = np.mean(x.gpu_times)
con_std[1,0] = np.std(x.cpu_times)
con_std[1,1] = np.std(x.gpu_times)
x = benchmark(sci_conv, (a512_1,b512_1,n_conv), n_repeat=n_rep)
con_res[2,0] = np.mean(x.cpu_times)
con_res[2,1] = np.mean(x.gpu_times)
con_std[2,0] = np.std(x.cpu_times)
con_std[2,1] = np.std(x.gpu_times)
x = benchmark(sci_conv, (a512_2,b512_2,n_conv), n_repeat=n_rep)
con_res[3,0] = np.mean(x.cpu_times)
con_res[3,1] = np.mean(x.gpu_times)
con_std[3,0] = np.std(x.cpu_times)
con_std[3,1] = np.std(x.gpu_times)
x = benchmark(sci_conv, (a512_3,b512_3,n_conv), n_repeat=n_rep)
con_res[4,0] = np.mean(x.cpu_times)
con_res[4,1] = np.mean(x.gpu_times)
con_std[4,0] = np.std(x.cpu_times)
con_std[4,1] = np.std(x.gpu_times)



sci_conv            :    CPU:89179.936 us   +/-1442.306 (min:85881.600 / max:95263.300) us     GPU-0:89932.776 us   +/-4249.736 (min:73931.778 / max:110384.125) us
sci_fftconv         :    CPU:89056.892 us   +/-1442.558 (min:84912.200 / max:92810.000) us     GPU-0:89440.856 us   +/-3596.935 (min:73424.896 / max:105223.167) us


In [ ]:
x = benchmark(sci_fftconv, (a128,b128,n_conv), n_repeat=n_rep)
con_res[0,2] = np.mean(x.cpu_times)
con_res[0,3] = np.mean(x.gpu_times)
con_std[0,2] = np.std(x.cpu_times)
con_std[0,3] = np.std(x.gpu_times)
x = benchmark(sci_fftconv, (a256,b256,n_conv), n_repeat=n_rep)
con_res[1,2] = np.mean(x.cpu_times)
con_res[1,3] = np.mean(x.gpu_times)
con_std[1,2] = np.std(x.cpu_times)
con_std[1,3] = np.std(x.gpu_times)
x = benchmark(sci_fftconv, (a512_1,b512_1,n_conv), n_repeat=n_rep)
con_res[2,2] = np.mean(x.cpu_times)
con_res[2,3] = np.mean(x.gpu_times)
con_std[2,2] = np.std(x.cpu_times)
con_std[2,3] = np.std(x.gpu_times)
x = benchmark(sci_fftconv, (a512_2,b512_2,n_conv), n_repeat=n_rep)
con_res[3,2] = np.mean(x.cpu_times)
con_res[3,3] = np.mean(x.gpu_times)
con_std[3,2] = np.std(x.cpu_times)
con_std[3,3] = np.std(x.gpu_times)
x = benchmark(sci_fftconv, (a512_3,b512_3,n_conv), n_repeat=n_rep)
con_res[4,2] = np.mean(x.cpu_times)
con_res[4,3] = np.mean(x.gpu_times)
con_std[4,2] = np.std(x.cpu_times)
con_std[4,3] = np.std(x.gpu_times)

In [10]:
r.cpu_times

array([0.4625065, 0.4666529])

## 1.2: Numba

* "Numba is designed to be used with NumPy arrays and functions" (https://numba.pydata.org/)
* "SciPy is already a high-performance library so in most cases I would expect numba to perform worse (or rarely: slightly better)" (https://stackoverflow.com/questions/55317665/numba-jit-with-scipy)

## 1.3: Numba-Scipy
* Convolution not among supported functions (https://numba-scipy.readthedocs.io/en/latest/reference/special.html)

In [ ]:
!pip install numba-scipy

## 1.4 Cupy

In [29]:
def cupy_conv(a,b, n=1):
    ac = cupy.array(a, dtype=cupy.float32)
    bc = cupy.array(b, dtype=cupy.float32)
    for _ in range(n):
        a = ndimage.convolve(ac, bc, mode='constant')
    return cupy.asnumpy(a)

In [32]:
n_rep = 3

x = benchmark(sci_fftconv, (a128,b128,n_conv), n_repeat=n_rep)
con_res[0,4] = np.mean(x.cpu_times)
con_res[0,5] = np.mean(x.gpu_times)
con_std[0,4] = np.std(x.cpu_times)
con_std[0,5] = np.std(x.gpu_times)
x = benchmark(sci_fftconv, (a256,b256,n_conv), n_repeat=n_rep)
con_res[1,4] = np.mean(x.cpu_times)
con_res[1,5] = np.mean(x.gpu_times)
con_std[1,4] = np.std(x.cpu_times)
con_std[1,5] = np.std(x.gpu_times)
x = benchmark(sci_fftconv, (a512_1,b512_1,n_conv), n_repeat=n_rep)
con_res[2,4] = np.mean(x.cpu_times)
con_res[2,5] = np.mean(x.gpu_times)
con_std[2,4] = np.std(x.cpu_times)
con_std[2,5] = np.std(x.gpu_times)
x = benchmark(sci_fftconv, (a512_2,b512_2,n_conv), n_repeat=n_rep)
con_res[3,4] = np.mean(x.cpu_times)
con_res[3,5] = np.mean(x.gpu_times)
con_std[3,4] = np.std(x.cpu_times)
con_std[3,5] = np.std(x.gpu_times)
x = benchmark(sci_fftconv, (a512_3,b512_3,n_conv), n_repeat=n_rep)
con_res[4,4] = np.mean(x.cpu_times)
con_res[4,5] = np.mean(x.gpu_times)
con_std[4,4] = np.std(x.cpu_times)
con_std[4,5] = np.std(x.gpu_times)

cupy_conv           :    CPU:24173928.120 us   +/-160777.839 (min:23962332.100 / max:24470096.800) us     GPU-0:24174355.859 us   +/-160776.394 (min:23962742.188 / max:24470523.438) us


In [35]:
%lprun -f cupy_conv cupy_conv(a,b, 105)

Timer unit: 1e-07 s

Total time: 24.0845 s
File: C:\Users\jo77pihe\AppData\Local\Temp/ipykernel_13704/3009679358.py
Function: cupy_conv at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def cupy_conv(a,b, n=1):
     2       106       1099.0     10.4      0.0      for _ in range(n):
     3       105     464168.0   4420.6      0.2          a = ndimage.convolve(cupy.array(a), cupy.array(b), mode='constant')
     4         1  240379695.0 240379695.0     99.8      return cupy.asnumpy(a)

## 1.5 PyTorch

In [17]:
import torch

In [30]:
def torch_conv(a,b, n=1):
    device = torch.device("cuda")
    torch.cuda.synchronize()
    with torch.no_grad():
        ac = torch.tensor(a[None, None, ...], dtype=torch.float32).pin_memory().to(device)
        bc = torch.torch.tensor(b[None,None,...], dtype=torch.float32).pin_memory().to(device)
        conv = torch.nn.Conv3d(1, 1, a.shape, stride=1, padding=(a.shape[0]//2,a.shape[1]//2,a.shape[2]//2), padding_mode='reflect',
                               bias=False, device=device, dtype=torch.float32)
        for _ in range(n):
            conv.weight = torch.nn.Parameter(bc)
            ac = conv(ac)
    # torch.cuda.synchronize()
    return (ac.detach().to(torch.device('cpu'))[0,0,:,:,:]).numpy()

In [27]:
n_rep = 3

x = benchmark(sci_fftconv, (a128,b128,n_conv), n_repeat=n_rep)
con_res[0,6] = np.mean(x.cpu_times)
con_res[0,7] = np.mean(x.gpu_times)
con_std[0,6] = np.std(x.cpu_times)
con_std[0,7] = np.std(x.gpu_times)
x = benchmark(sci_fftconv, (a256,b256,n_conv), n_repeat=n_rep)
con_res[1,6] = np.mean(x.cpu_times)
con_res[1,7] = np.mean(x.gpu_times)
con_std[1,6] = np.std(x.cpu_times)
con_std[1,7] = np.std(x.gpu_times)
x = benchmark(sci_fftconv, (a512_1,b512_1,n_conv), n_repeat=n_rep)
con_res[2,6] = np.mean(x.cpu_times)
con_res[2,7] = np.mean(x.gpu_times)
con_std[2,6] = np.std(x.cpu_times)
con_std[2,7] = np.std(x.gpu_times)
x = benchmark(sci_fftconv, (a512_2,b512_2,n_conv), n_repeat=n_rep)
con_res[3,6] = np.mean(x.cpu_times)
con_res[3,7] = np.mean(x.gpu_times)
con_std[3,6] = np.std(x.cpu_times)
con_std[3,7] = np.std(x.gpu_times)
x = benchmark(sci_fftconv, (a512_3,b512_3,n_conv), n_repeat=n_rep)
con_res[4,6] = np.mean(x.cpu_times)
con_res[4,7] = np.mean(x.gpu_times)
con_std[4,6] = np.std(x.cpu_times)
con_std[4,7] = np.std(x.gpu_times)

torch_conv          :    CPU:3853818.720 us   +/-63086.658 (min:3785164.900 / max:3984520.200) us     GPU-0:3853953.833 us   +/-63080.924 (min:3785316.406 / max:3984621.826) us


In [ ]:
%lprun -f torch_conv torch_conv(a,b, 105)

Timer unit: 1e-07 s

Total time: 3.72042 s
File: C:\Users\jo77pihe\AppData\Local\Temp/ipykernel_13704/1315201415.py
Function: torch_conv at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def torch_conv(a,b, n=1):
     2         1        183.0    183.0      0.0      device = torch.device("cuda")
     3         1       1378.0   1378.0      0.0      torch.cuda.synchronize()
     4         1        299.0    299.0      0.0      with torch.no_grad():
     5         1      10791.0  10791.0      0.0          ac = torch.tensor(a[None, None, ...], dtype=torch.float32).pin_memory().to(device)
     6         1       5711.0   5711.0      0.0          bc = torch.torch.tensor(b[None,None,...], dtype=torch.float32).pin_memory().to(device)
     7         2      11652.0   5826.0      0.0          conv = torch.nn.Conv3d(1, 1, a.shape, stride=1, padding=(a.shape[0]//2,a.shape[1]//2,a.shape[2]//2), padding_mode='reflect',
     8         1         29.0     29.0      0.0                                 bias=False, device=device, dtype=torch.float32)
     9         1        933.0    933.0      0.0          conv.weight = torch.nn.Parameter(bc)
    10         1       9835.0   9835.0      0.0          ac = conv(ac)
    11                                               # torch.cuda.synchronize()
    12         1   37163348.0 37163348.0     99.9      return (ac.detach().to(torch.device('cpu'))[0,0,:,:,:]).numpy()

# 2 Write own function and speed up

## 2.0 Cython 

In [ ]:
def cython_conv(a,b, n = 1):
    for _ in range(n):
        a = cc.myconvolve(a,b))
    return a

In [ ]:
n_rep = 3

x = benchmark(sci_fftconv, (a128,b128,n_conv), n_repeat=n_rep)
con_res[0,8] = np.mean(x.cpu_times)
con_res[0,9] = np.mean(x.gpu_times)
con_std[0,8] = np.std(x.cpu_times)
con_std[0,9] = np.std(x.gpu_times)
x = benchmark(sci_fftconv, (a256,b256,n_conv), n_repeat=n_rep)
con_res[1,8] = np.mean(x.cpu_times)
con_res[1,9] = np.mean(x.gpu_times)
con_std[1,8] = np.std(x.cpu_times)
con_std[1,9] = np.std(x.gpu_times)
x = benchmark(sci_fftconv, (a512_1,b512_1,n_conv), n_repeat=n_rep)
con_res[2,8] = np.mean(x.cpu_times)
con_res[2,9] = np.mean(x.gpu_times)
con_std[2,8] = np.std(x.cpu_times)
con_std[2,9] = np.std(x.gpu_times)
x = benchmark(sci_fftconv, (a512_2,b512_2,n_conv), n_repeat=n_rep)
con_res[3,8] = np.mean(x.cpu_times)
con_res[3,9] = np.mean(x.gpu_times)
con_std[3,8] = np.std(x.cpu_times)
con_std[3,9] = np.std(x.gpu_times)
x = benchmark(sci_fftconv, (a512_3,b512_3,n_conv), n_repeat=n_rep)
con_res[4,8] = np.mean(x.cpu_times)
con_res[4,9] = np.mean(x.gpu_times)
con_std[4,8] = np.std(x.cpu_times)
con_std[4,9] = np.std(x.gpu_times)

## 2.1 Numba

In [ ]:
def num_c(a, b):
    (NAx, NAy, NAz) = a.shape
    (NBx, NBy, NBz) = a.shape
    Deg = NAz +NBz -1
    C = np.zeros((NAx, NBy, Deg))

    assert((NAx == NBx) and (NAy == NBy))

    for n in range(0, (Deg)):
        for m in range(0, n + 1):
            if ((n - m) < NAz and m < NBz):
                for i in range(0, NAx):
                    for j in range(0, NAy):
                        C[i, j, n] = C[i, j, n] + a[i, j, (n - m)] * b[i, j, m]
    return C

In [ ]:
def numba_conv(a,b, n = 1):
    for _ in range(n):
        a = num_c(a,b)
    return a

In [ ]:
n_rep = 3

x = benchmark(sci_fftconv, (a128,b128,n_conv), n_repeat=n_rep)
con_res[0,10] = np.mean(x.cpu_times)
con_res[0,11] = np.mean(x.gpu_times)
con_std[0,10] = np.std(x.cpu_times)
con_std[0,11] = np.std(x.gpu_times)
x = benchmark(sci_fftconv, (a256,b256,n_conv), n_repeat=n_rep)
con_res[1,10] = np.mean(x.cpu_times)
con_res[1,11] = np.mean(x.gpu_times)
con_std[1,10] = np.std(x.cpu_times)
con_std[1,11] = np.std(x.gpu_times)
x = benchmark(sci_fftconv, (a512_1,b512_1,n_conv), n_repeat=n_rep)
con_res[2,10] = np.mean(x.cpu_times)
con_res[2,11] = np.mean(x.gpu_times)
con_std[2,10] = np.std(x.cpu_times)
con_std[2,11] = np.std(x.gpu_times)
x = benchmark(sci_fftconv, (a512_2,b512_2,n_conv), n_repeat=n_rep)
con_res[3,10] = np.mean(x.cpu_times)
con_res[3,11] = np.mean(x.gpu_times)
con_std[3,10] = np.std(x.cpu_times)
con_std[3,11] = np.std(x.gpu_times)
x = benchmark(sci_fftconv, (a512_3,b512_3,n_conv), n_repeat=n_rep)
con_res[4,10] = np.mean(x.cpu_times)
con_res[4,11] = np.mean(x.gpu_times)
con_std[4,10] = np.std(x.cpu_times)
con_std[4,11] = np.std(x.gpu_times)